Steps:
- Scarping Job postings with Selenium and bs4
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [150]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn


Select wanted url

In [151]:
city = str(input('Please enter the city you want to work in'))
state = str(input('Please enter the state you want to work in'))
country = str(input('Please enter the country you want to work in'))
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location='+ city + '%2C%20'+ state +'%2C%20'+ country
url2 = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Geneva%2C%20Geneva%2C%20Switzerland&geoId=107814425&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0'
no_of_jobs = 50

url



'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Zürich%2C%20Zurich%2C%20Switzerland'

open windows with url stated above in

In [152]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [153]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [154]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [155]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Business Intelligence Analyst (80%)
Spitex Zürich Limmat


Create job container

In [156]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

48


create for loop to look for all jobs

In [157]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 48 jobs.


In [158]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [159]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

{'Title': ['Business Intelligence Analyst (80%)', 'Data Analyst', 'Data Engineer (80 - 100%)', 'Business Analyst (m/w), Zurich', 'Global Digital Data Analyst', 'Data Scientist - Zurich', 'Data Scientist 100% (w/m) 100%', 'Data Analyst (BI / Digital Analytics) (w/m) 80-100%', 'Data Scientist', 'Data Engineer', 'Data Analyst (temporary)', 'Product Analyst, Calendar and Tasks', 'Business Analyst', 'HR Data Administrator & Analyst (50% - 60%)', 'Business Analyst 100% (m/w)', 'Data Consultant', 'Data Scientist - QuantumBlack', 'Data Engineer', 'Business/Data Analyst Basel IV #154939', 'Analytics Engineer (m/f/d)', 'Business Analyst', 'Reporting/ Data Analyst (Banking - Operations)', 'Digital Analyst (w/m/d)', 'Data Scientist Personalisierung - Zürich', 'Junior Business Analyst', 'Data scientist for AI and Monitoring Models Validation', 'Data Science Analyst (AA/AI) 100% (f/m/d)', 'Business Analyst', 'Business Insights Analyst, Direct-to-Consumer - Global', 'Business Analyst 80 - 100%', 'Dat

In [160]:
df = pd.DataFrame(ml)
df.head(1)

,Title,Company,Locations,Post Date
0,Business Intelligence Analyst (80%),Spitex Zürich Limmat,"Zurich, Zurich, Switzerland",2021-01-22


Separate City from State and Country


In [161]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [162]:
new2 = new.rename(columns = {0: 'City2'})

In [163]:
new2


,City2,1,2
0,Zurich,Zurich,Switzerland
1,Zurich,Zurich,Switzerland
2,Zurich,Zurich,Switzerland
3,Schlatt (Zürich),Zurich,Switzerland
4,Zurich,Switzerland,None
5,Zurich,Switzerland,None
6,Horgen,Zurich,Switzerland
7,Zurich,Zurich,Switzerland
8,Zurich,Switzerland,None
9,Zurich,Switzerland,None


merge df and new, drop Locations as it is redundant now

In [164]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

,Title,Company,Post Date,City2
0,Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
1,Data Analyst,EPAM Systems,2021-01-15,Zurich
2,Data Engineer (80 - 100%),Swiss Re,2021-01-16,Zurich
3,"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt (Zürich)


In [165]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new5 = new4.City
new5.head()

0     Zurich
1     Zurich
2     Zurich
3    Schlatt
4     Zurich
Name: City, dtype: object

In [166]:
df4 = df2.join(new5, how = 'left')
df4.drop(columns = ["City2"], inplace = True)
df4.set_index('Title', inplace = True)
df4

,Company,Post Date,City
Title,,,
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
Data Analyst,EPAM Systems,2021-01-15,Zurich
Data Engineer (80 - 100%),Swiss Re,2021-01-16,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Data Scientist - Zurich,Freshtechit,2021-01-21,Zurich
Data Scientist 100% (w/m) 100%,Properti AG,2021-01-10,Horgen
Data Analyst (BI / Digital Analytics) (w/m) 80-100%,NZZ,2021-01-17,Zurich
Data Scientist,Tata Consultancy Services,2021-01-14,Zurich


create CSV, with todays Date

In [176]:
date_object = datetime.date.today()
date = str(date_object) + '_'
date

df4.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/' + date + city + '_' + country + '.csv')

concat new jobs to existing list

In [168]:
final_df = pd.read_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')
final_df.set_index('Title', inplace = True)
final_df.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


In [169]:
dfWIP = pd.concat([final_df, df4], axis=0)
dfWIP.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


drop duplicated rows

In [172]:
new_final_df = dfWIP.drop_duplicates()
new_final_df

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt
"Product Analyst, Calendar and Tasks",Google,2021-01-08,Schlatt
Reporting/ Data Analyst (Banking - Operations),Experis Switzerland,2021-01-12,Zurich
Data Scientist Intern - QuantumBlack,QuantumBlack,2021-01-19,Zurich
Junior Business Analyst,SIX,2021-01-15,Zurich


In [171]:
new_final_df.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')

